In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import meshio
import os
import time
from scipy.interpolate import griddata
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import mean_squared_error


# Loading Dataset

In [ ]:
x = np.arange(0.006, 0.0135, (0.0135-0.006)/300)
y = np.arange(0, 0.0025, 0.0025/75)

data_path = './Data'
path_sep = '\\' # use '/' for Unix and '\\' for Windows
folders = os.listdir(data_path)
        
subfolder = []
path = []
names = []
condition = []

for i in folders:
    if os.path.isdir(data_path + path_sep + i): 
        subfolder.append(data_path + path_sep + i)

for folder in subfolder:
    files = os.listdir(folder + path_sep)
    for i in files:
        ext = os.path.splitext(i)
        if (ext[-1].lower() == '.vtk') & (ext[0][-2] != '_'):
            names.append(ext[0])
            string = ext[0].replace('ER', '').replace('Tin', '').replace('Uin', '').replace('Twall', '').split('_')[0:4]
            var = []
            for j in string:
                if j == 'Adiabatic':
                    var.append(0.)
                else:
                    var.append(float(j))
            condition.append(var)

Q_list = []
for folder in subfolder:
    files = os.listdir(folder + path_sep)
    for counter,file in enumerate(files):
        mesh = meshio.read(folder+ path_sep +file)
        points = mesh.points
        Qdot = mesh.point_data['Qdot']
        boolArr = (points[:,1] == 0) & (points[:,0] >= 0.006)  
        Qdot = Qdot[boolArr]
        points = points[boolArr]
        old_points = points[:,[0, 2]]
        grid_x, grid_y = np.meshgrid(x, y)
        grid_new = griddata(old_points, Qdot, (grid_x, grid_y), method='nearest')
        Q_list.append(grid_new)
Q_list

# Data Preparation

In [ ]:
Qdot = np.array(Q_list)
indices = [10, 88, 100]
# OutLier
fig, axs = plt.subplots(len(indices),1,figsize=(15, 7))
# Equivalence ratio (1.0)
# Temperature (460)
# Velocity (0.50 m/s)
# Wall Temperature (373 or None)
for idx, i in enumerate(indices):
    img = Qdot[i]
    axs[idx].imshow(img)
    # axs[idx].set_title(f'Equivalence ratio: {condition[i][0] * 0.001:.2f} '
    #                    f'Temperature: {condition[i][1]} '
    #                    f'Velocity: {condition[i][2] * 0.01:.2f} '
    #                    f'Wall Temperature: {condition[i][3]}')
    axs[idx].axis('off')  
plt.tight_layout()
plt.show()

In [4]:
Qdot = np.delete(Qdot, indices, axis=0)
condition = np.delete(condition, indices, axis=0)

# Data Normalization
Qdot = Qdot / np.max(Qdot)
mean = Qdot.mean(axis = 0)
Qdot = np.reshape(Qdot, (-1, 75, 300, 1))

# Label Normalization
normaliser = []
conditions = np.array(condition)
df = np.zeros(conditions.shape)
for i in range(conditions.shape[1]):
    df[:,i] = conditions[:,i] / np.max(conditions[:,i])
    normaliser.append(np.max(conditions[:,i]))

# Data split
train_data, test_data, label_train, label_test = train_test_split(Qdot, df, test_size = 0.15, shuffle=True)

In [5]:
# Data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.05, 
    height_shift_range=0.05,     
    zoom_range=0.1,   
    fill_mode='nearest'  
)

train_data_augmented = []
label_train_augmented = []
for i in range(train_data.shape[0]):
    img = train_data[i].reshape((1, 75, 300, 1)) 
    it = datagen.flow(img, batch_size=1)
    
    for _ in range(5):
        batch = it.next()
        train_data_augmented.append(batch[0])
        label_train_augmented.append(label_train[i])     
        
train_data_augmented = np.array(train_data_augmented)
label_train_augmented = np.array(label_train_augmented)

In [ ]:
#print the input data
#print out the condition data for that image as well
for i in range(236//20):
    fig = plt.figure(figsize=(16, 80))
    plt.imshow(np.reshape(Qdot[i*10],(75, 300)))
    plt.axis('off')
    plt.title(f'Equivalence ratio: {condition[i][0] * 0.001:.2f} '
                       f'Temperature: {condition[i][1]} '
                       f'Velocity: {condition[i][2] * 0.01:.2f} '
                       f'Wall Temperature: {condition[i][3]}')


In [ ]:
fig = plt.figure(figsize=(20, 40))
columns = 6
rows = 40

for i in range(1, columns * rows):
    img = Qdot[(i-1)]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.title(f"Image {i}")  
    plt.axis('off')
plt.show()

# Outlier imgs: 11 89 101

# Model define 


## Convolutional Autoencoder

In [7]:
class ConvAE(tf.keras.Model):
  def __init__(self, latent_dim):
    super(ConvAE, self).__init__()
    self.latent_dim = latent_dim
    
    self.encoder = tf.keras.Sequential([
      # Input layer 
      tf.keras.layers.InputLayer(input_shape = (75,300,1)),
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
      # tf.keras.layers.Dropout(0.3),       
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(5, 5)),
      # tf.keras.layers.Dropout(0.3),         
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(5, 5)),
      # tf.keras.layers.Dropout(0.3),   
          
      # Flatten  
      tf.keras.layers.Flatten(),
      
      # Fully NN --> Laten variables
      tf.keras.layers.Dense(latent_dim)
    ])
    
    self.decoder = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape = (latent_dim,)),
      
      # Fully NN 
      tf.keras.layers.Dense(units = 1*4*64),
      tf.keras.layers.Reshape(target_shape = (1,4,64)),
      
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=5, strides=5,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),     
      
       # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),     

      # Conv layer  
      tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=5, strides=5,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      # Conv layer  
      tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
                            
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, strides=3, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, strides=1, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1,padding='same', activation='sigmoid'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

# CAE.encoder.summary()
# CAE.decoder.summary()

## Variantional autoencoder

- Encoder: change output to mean and log variance
- Decoder: add loss function Kullback–Leibler divergence term

In [10]:
class variational_autoencoder(tf.keras.Model):
  def __init__(self, latent_dim):
    super(variational_autoencoder, self).__init__()
    self.latent_dim = latent_dim
    
    self.encoder = tf.keras.Sequential([
      # Input layer 
      tf.keras.layers.InputLayer(input_shape = (75,300,1)),
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
      # tf.keras.layers.Dropout(0.3),       
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(5, 5)),
      # tf.keras.layers.Dropout(0.3),         
      
      # Conv layer + BatchNom + LeakyReLU + MaxPooling
      tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.MaxPooling2D(pool_size=(5, 5)),
      # tf.keras.layers.Dropout(0.3),   
          
      # Flatten  
      tf.keras.layers.Flatten(),
      
      # Fully NN --> Laten variables
      tf.keras.layers.Dense(latent_dim*2)
    ])
    
    self.decoder = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape = (latent_dim,)),
      
      # Fully NN 
      tf.keras.layers.Dense(units = 1*4*64),
      tf.keras.layers.Reshape(target_shape = (1,4,64)),
      
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=5, strides=5,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),     
      
       # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),     

      # Conv layer  
      tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=5, strides=5,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      # Conv layer  
      tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=5, strides=1,padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
                            
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, strides=3, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      # Conv layer 
      tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, strides=1, padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.2),
      
      tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1,padding='same', activation='sigmoid'),
    ])

  def sample(self, mean, log_var):
      epsilon = tf.random.normal(shape=tf.shape(mean))
      return mean + tf.exp(0.5 * log_var) * epsilon

  def call(self, x):

      # Encode to get mean and log variance
      z_mean_log_var = self.encoder(x)
      z_mean, z_log_var = tf.split(z_mean_log_var, num_or_size_splits=2, axis=1)
      
      # Reparameterization trick to sample latent space
      z = self.sample(z_mean, z_log_var)
      
      # Decode to reconstruct
      reconstructed = self.decoder(z)
      
      # KL divergence loss
      beta = 0.1
      kl_loss = beta * (-0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
      # tf.print("KL Loss:", tf.reduce_mean(kl_loss)) 
    
      # Add KL divergence loss to the total loss
      self.add_loss(tf.reduce_mean(kl_loss))  
      
      return reconstructed
    
latent_dim = 8
VAE_aug = variational_autoencoder(latent_dim)
VAE_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy')

# VAE.encoder.summary()
# VAE.decoder.summary()

# Model training

In [ ]:
# CAE
Clatent_dim = 8
CAE = ConvAE(latent_dim)
CAE.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), loss='mse')
hist_cae = CAE.fit(train_data, train_data,epochs=200,batch_size = 8,validation_data=(test_data, test_data))

# Save model for next analyse 
CAE.save("Trian_5") ##Important!!

In [ ]:
# Augmentation
latent_dim = 8
AUG = ConvAE(latent_dim)
AUG.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), loss='mse')
hist_cae_aug = AUG.fit(train_data_augmented, train_data_augmented,epochs=200,batch_size = 8,validation_data=(test_data, test_data))

# Save model for next analyse 
AUG.save('cae_aug_bs8_lr1e-4_latent_4') ##Important!!

In [ ]:
# VAE
hist_vae = VAE_aug.fit(train_data_augmented, train_data_augmented, epochs=200, batch_size=8, validation_data=(test_data, test_data))

## Results

In [ ]:
# Results CAE_aug
loss_aug = hist_cae_aug.history['loss']
val_loss_aug = hist_cae_aug.history['val_loss']

plt.semilogy(loss_aug, label ='Train loss')
plt.semilogy(val_loss_aug, label = 'Validation loss')
# plt.plot(loss, label ='Train loss')
# plt.plot(val_loss, label = 'Validation loss')
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.title(f'CAE Latent dimension: {latent_dim}')
plt.legend()
plt.show()

In [ ]:
# Results CAE
hist_cae = tf.keras.models.load_model("Models/cae/cae_aug_bs8_lr1e-4_latent8")
loss = hist_cae.history['loss']
val_loss = hist_cae.history['val_loss']

plt.semilogy(loss, label ='Train loss')
plt.semilogy(val_loss, label = 'Validation loss')
# plt.plot(loss, label ='Train loss')
# plt.plot(val_loss, label = 'Validation loss')
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.title(f'CAE Latent dimension: {latent_dim}')
plt.legend()
plt.show()

In [ ]:
# Results VAE
loss = hist_vae.history['loss']
val_loss = hist_vae.history['val_loss']

plt.semilogy(loss, label ='Train loss')
plt.semilogy(val_loss, label = 'Validation loss')
# plt.plot(loss, label ='Train loss')
# plt.plot(val_loss, label = 'Validation loss')
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.title(f'VAE Latent dimension: {latent_dim}')
plt.legend()
plt.show()

## Visualization

### CAE

In [ ]:
## CAE Visualization
encoded_test_cae = CAE.encoder(test_data).numpy()
decoded_test_cae = CAE.decoder(encoded_test_cae).numpy()

# loops = 9
# images_per_loop = 4

fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 9
for i in range(1, columns * rows + 1, 2):
  img = test_data[i - 1] 
  ax = fig.add_subplot(rows, columns, i)  
  plt.imshow(img)
  plt.title("original")
  plt.axis('off')  

  decoded_img = decoded_test_cae[i - 1]  
  ax = fig.add_subplot(rows, columns, i + 1) 
  plt.imshow(decoded_img)
  plt.title("CAE reconstructed")
  plt.axis('off')  

plt.show()

In [ ]:
AUG = tf.keras.models.load_model('Models/cae/cae_aug_bs8_lr1e-4_latent8') 
AUG4 = tf.keras.models.load_model('Models/cae/cae_aug_bs8_lr1e-4_latent4') 

encoded_cae_aug = AUG.encoder(test_data).numpy()
decoded_cae_aug = AUG.decoder(encoded_cae_aug).numpy()

encoded_cae_aug4 = AUG4.encoder(test_data).numpy()
decoded_cae_aug4 = AUG4.decoder(encoded_cae_aug4).numpy()

fig = plt.figure(figsize=(10, 22))  
columns = 3
rows = 20  

for i in range(1, rows * columns + 1, 3):
    idx = (i - 1) // 3  

    # original
    img = test_data[idx]
    ax = fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.title("Original")
    plt.axis('off')

    # CAE reconstructed (latent 8)
    decoded_img = decoded_cae_aug[idx]
    ax = fig.add_subplot(rows, columns, i + 1)
    plt.imshow(decoded_img)
    plt.title("Latent 8")
    plt.axis('off')

    # CAE reconstructed (latent 4)
    decoded_img4 = decoded_cae_aug4[idx]
    ax = fig.add_subplot(rows, columns, i + 2)
    plt.imshow(decoded_img4)
    plt.title("Latent 4")
    plt.axis('off')

plt.show()


### VAE

In [ ]:
# VAE visualization
# latent = autoencoder.encoder(test_data).numpy()
encoded_test_vae = VAE_aug.encoder(test_data).numpy()
mean_vae,log_var = tf.split(encoded_test_vae, num_or_size_splits=2, axis=1)
decoded_test_vae = VAE_aug.decoder(VAE_aug.sample(mean_vae,log_var)).numpy()

# loops = 9
# images_per_loop = 4
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 9
for i in range(1, columns * rows + 1, 2):
  img = test_data[i - 1] 
  ax = fig.add_subplot(rows, columns, i)  
  plt.imshow(img)
  plt.title("original")
  plt.axis('off')  

  decoded_img = decoded_test_vae[i - 1]  
  ax = fig.add_subplot(rows, columns, i + 1) 
  plt.imshow(decoded_img)
  plt.title("VAE reconstructed")
  plt.axis('off')  

plt.show()
# fig, axs = plt.subplots(loops * 2, images_per_loop, figsize=(images_per_loop * 2, loops * 2))

# for loop in range(loops):
#     for i in range(images_per_loop):
#         index = loop * images_per_loop + i
        
#         axs[2 * loop, i].imshow(test_data[index].squeeze())
#         axs[2 * loop, i].axis('off')
#         axs[2 * loop, i].set_title("Original")

#         axs[2 * loop + 1, i].imshow(decoded_test_vae[index].squeeze())
#         axs[2 * loop + 1, i].axis('off')
#         axs[2 * loop + 1, i].set_title("VAE Reconstructed")

# plt.tight_layout()
# plt.show()



# Fully Connected Neural Network

In [ ]:
param_dim = conditions.shape[1]
latent_dim = 8
num_layers = 8 

mapping = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(param_dim,)) 
] + [tf.keras.layers.Dense(2**(n+2), activation='relu') for n in range(1, num_layers + 1)  
] + [
    tf.keras.layers.Dense(latent_dim, activation='linear')  
])
mapping.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')

encoded_train_mapping = AUG.encoder(train_data).numpy()
encoded_test_mapping = AUG.encoder(test_data).numpy()

hist_mapping = mapping.fit(label_train, encoded_train_mapping, epochs=1000, batch_size=8,validation_data=(label_test,encoded_test_mapping))
# mapping.save('fcnn_bs8_lr1e-4_latent_8_nlayers_3')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Prefix for model file path
model_dir = 'Models/cae/'
model_prefix = 'Train_'

# Range of model layers
n_counts = range(1, 7)

# List to store MSE values for each model
mse_values = []

# Loop through the models and compute MSE
for n in n_counts:
    model_path = os.path.join(model_dir, f'{model_prefix}{n}')
    
    # Load model and predict
    model = tf.keras.models.load_model(model_path)
    predictions = model.predict(test_data)
    
    # Flatten the data for MSE calculation
    test_data_flat = test_data.reshape(test_data.shape[0], -1)
    predictions_flat = predictions.reshape(predictions.shape[0], -1)

    # Compute and store MSE
    mse = mean_squared_error(test_data_flat, predictions_flat)
    mse_values.append(mse)
    
    print(f'Model with {n}. Train - MSE: {mse}')

# Convert mse_values to numpy array for easier manipulation
mse_values = np.array(mse_values)

# Plotting
plt.figure(figsize=(10, 7))
plt.style.use('ggplot')
# Plot MSE values with semilog scale, customized line style and markers
plt.semilogy(n_counts, mse_values, marker='o', linestyle='-', color='b', markersize=10, linewidth=2)

# Title and labels with font size adjustments
plt.title('Comparison of Model MSE with different hyperparameters', fontsize=16)
plt.xlabel('Train step', fontsize=14)
plt.ylabel('Mean Squared Error (MSE)', fontsize=14)

# Customizing tick sizes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Grid customization
plt.grid(True, linestyle='--', linewidth=0.7)

# Add shaded region for simulated error bounds
mse_min = mse_values * 0.9  # Lower bound
mse_max = mse_values * 1.1  # Upper bound
plt.fill_between(n_counts, mse_min, mse_max, color='b', alpha=0.2)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# fcnn4 = tf.keras.models.load_model("Models/fcnn/fcnn_bs8_lr1e-4_latent_4_nlayers_8")
# fcnn8 = tf.keras.models.load_model("Models/fcnn/fcnn_bs8_lr1e-4_latent_8_nlayers_8")


loss_map = hist_mapping.history['loss']
val_loss_map = hist_mapping.history['val_loss']

# plt.semilogy(loss_map, label ='Train loss')
# plt.semilogy(val_loss_map, label = 'Validation loss')
plt.plot(loss_map, label ='Train loss')
plt.plot(val_loss_map, label = 'Validation loss')
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.title(f'FCNN Latent dimention: {latent_dim}')
plt.legend()
plt.show()

In [ ]:
fcnn = tf.keras.models.load_model("Models/fcnn/fcnn_bs8_lr1e-4_latent_8_nlayers_8")
fcnn4 = tf.keras.models.load_model("Models/fcnn/fcnn_bs8_lr1e-4_latent_4_nlayers_8")

idx = 137
# try 19, 32, 100, 170, 230

# Select parameters for the specific index
new_params = df[idx]
params = np.expand_dims(new_params, axis=0)

# Get the latent data and decoded images from latent dimensions 8 and 4
latent_data_8 = fcnn(params)  # Assuming this is for latent dimension 8
pred_img_8 = (AUG.decoder(latent_data_8)).numpy()

latent_data_4 = fcnn4(params)  # Assuming this is for latent dimension 4
pred_img_4 = (AUG4.decoder(latent_data_4)).numpy()

# Plot the original, latent 8 and latent 4 decoded images
fig = plt.figure(figsize=(10, 8))

# Plot original image in the first row
img = Qdot[idx]
# Row 1: Original image
ax = fig.add_subplot(3, 1, 1)
plt.imshow(np.squeeze(img))
plt.axis('off')
plt.title(f'Equivalence ratio: {condition[idx][0] * 0.001:.2f} '
                       f'Temperature: {condition[idx][1]} '
                       f'Velocity: {condition[idx][2] * 0.01:.2f} '
                       f'Wall Temperature: {condition[idx][3]}')
# Row 2: Latent 8 image
ax = fig.add_subplot(3, 1, 2)
plt.imshow(np.squeeze(pred_img_8))
plt.axis('off')
# plt.title("Predicted Image from Latent 8", pad=10)

# Row 3: Latent 4 image
ax = fig.add_subplot(3, 1, 3)
plt.imshow(np.squeeze(pred_img_4))
plt.axis('off')
# plt.title("Predicted Image from Latent 4", pad=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
